![Logo](img/datasciencelab.png)

# dbm-16-hr-competion
#### Alexander Kopp & Leonhard Kühne-Hellmessen

1. Business Understanding
2. Data Understanding
3. Data Preperation  
**4. Modelling** 
5. Evaluation
6. Deployment

![modelling](img/modeling.png)

### 4. Modelling

In [1]:
# Load standard libraries
import numpy as np
import scipy.stats as stats
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#### Import cleaned data from EDA

In [2]:
df = pd.read_pickle('data/hr_train_clean.pkl')

#### Feature selection

In [3]:
df.head()

,id,satisfaction_level,last_evaluation,number_project,average_monthly_hours,time_spend_company,work_accident,left,promotion_last_5years,department,salary
0,0,0.65,0.96,5,226,2,1,0,0,marketing,medium
1,1,0.88,0.80,3,166,2,0,0,0,IT,low
2,2,0.69,0.98,3,214,2,0,0,0,sales,low
3,3,0.41,0.47,2,154,3,0,1,0,sales,low
4,4,0.87,0.76,5,254,2,1,0,0,hr,low


In [4]:
# Transfroming salary into integer (salary_int)
def varianten_salary(value):
    if value == "low" :
        return 1
    elif value == "medium":
        return 2
    else:
        return 3

df['salary_int'] = df.apply(lambda row: varianten_salary(row['salary']), axis=1)

In [5]:
df.head()

,id,satisfaction_level,last_evaluation,number_project,average_monthly_hours,time_spend_company,work_accident,left,promotion_last_5years,department,salary,salary_int
0,0,0.65,0.96,5,226,2,1,0,0,marketing,medium,2
1,1,0.88,0.80,3,166,2,0,0,0,IT,low,1
2,2,0.69,0.98,3,214,2,0,0,0,sales,low,1
3,3,0.41,0.47,2,154,3,0,1,0,sales,low,1
4,4,0.87,0.76,5,254,2,1,0,0,hr,low,1


In [6]:
# Remove unnecessary columns
df = df.drop(['id','department', 'work_accident', 'promotion_last_5years', 'salary'], axis=1)

In [7]:
df.head()

,satisfaction_level,last_evaluation,number_project,average_monthly_hours,time_spend_company,left,salary_int
0,0.65,0.96,5,226,2,0,2
1,0.88,0.80,3,166,2,0,1
2,0.69,0.98,3,214,2,0,1
3,0.41,0.47,2,154,3,1,1
4,0.87,0.76,5,254,2,0,1


#### Data preparation - feature engineering

In [8]:
# Enrich column'left' with data
df_num = pd.get_dummies(df.drop('left', axis=1)).join(df[['left']])

#### Verwendung von Scikit Learn (sklearn)

In [9]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold

In [10]:
# Set target vector y:
y = df_num['left'].values

# Set class label:
class_names = np.unique(y)

# Set characteristic names:
feature_names = np.array(['satisfaction_level'
    , 'last_evaluation'
    , 'number_project'
    , 'average_monthly_hours'
    , 'time_spend_company'
    , 'salary_int'
                         ])
# Create feature matrix:
x = df_num[feature_names].values

Split training data into internal test and training data sets:

In [11]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

#### 4.2.1 Decision Tree

Decision Trees (DTs) are a non-parametric supervised learning method used for classification and regression. The goal is to create a model that predicts the value of a target variable by learning simple decision rules inferred from the data features.

In [12]:
# DecisionTreeClassifier is a class capable of performing multi-class classification on a dataset
clf = DecisionTreeClassifier()
cv_scores = cross_val_score(clf, x_train, y_train, cv=10)
np.mean(cv_scores), np.std(cv_scores)

(0.97066593279869517, 0.005270645602250058)

### 4.2.2 Esemble methods
The goal of ensemble methods is to combine the predictions of several base estimators built with a given learning algorithm in order to improve generalizability / robustness over a single estimator.
We will have look at the two different families of ensemble methods, which are usually considered:
  1. In **averaging methods**, the driving principle is to build several estimators independently and then to average their predictions. On average, the combined estimator is usually better than any of the single base estimator because its variance is reduced.  
  *We will use **Random Forests** & **Extremely Randomized Trees***
  2. In **boosting methods**, base estimators are built sequentially and one tries to reduce the bias of the combined estimator. The motivation is to combine several weak models to produce a powerful ensemble.  
  *We will use **GradiantBoost** and **AdaBoost***

#### 4.2.2.1 Random Forests
In random forests, each tree in the ensemble is built from a sample drawn with replacement (i.e., a bootstrap sample) from the training set. In addition, when splitting a node during the construction of the tree, the split that is chosen is no longer the best split among all features. Instead, the split that is picked is the best split among a random subset of the features. As a result of this randomness, the bias of the forest usually slightly increases (with respect to the bias of a single non-random tree) but, due to averaging, its variance also decreases, usually more than compensating for the increase in bias, hence yielding an overall better model.

In [13]:
rfc = RandomForestClassifier()
cv_scores = cross_val_score(rfc, x_train, y_train, cv=10)
np.mean(cv_scores)

0.98506541060517461

In [14]:
rfc2 = RandomForestClassifier(n_estimators=100)
cv_scores = cross_val_score(rfc2, x_train, y_train, cv=10)
np.mean(cv_scores)

0.98706523448634265

In [15]:
rfc3 = RandomForestClassifier(n_estimators=250, min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0)
cv_scores = cross_val_score(rfc3, x_train, y_train, cv=10)
np.mean(cv_scores)

0.98679856781967601

#### 4.2.2.2 Extremely Randomized Trees
In extremely randomized trees, randomness goes one step further in the way splits are computed. As in random forests, a random subset of candidate features is used, but instead of looking for the most discriminative thresholds, thresholds are drawn at random for each candidate feature and the best of these randomly-generated thresholds is picked as the splitting rule. This usually allows to reduce the variance of the model a bit more, at the expense of a slightly greater increase in bias.

In [16]:
from sklearn.ensemble import ExtraTreesClassifier
etc = ExtraTreesClassifier(n_estimators=100, max_depth=None, min_samples_split=2, random_state=42)
scores = cross_val_score(etc, x_train, y_train, cv=10)
scores.mean()

0.98453261036908513

#### 4.2.2.3 AdaBoost
The core principle of AdaBoost is to fit a sequence of weak learners (i.e., models that are only slightly better than random guessing, such as small decision trees) on repeatedly modified versions of the data. The predictions from all of them are then combined through a weighted majority vote (or sum) to produce the final prediction

In [17]:
abc = AdaBoostClassifier(base_estimator=rfc2)
scores = cross_val_score(abc, x_train, y_train, cv=10)
scores.mean()

0.98706559051660536

In [18]:
abc.fit(x_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          learning_rate=1.0, n_estimators=50, random_state=None)

#### 4.2.2.4 GradiantBoost
Gradient Tree Boosting or Gradient Boosted Regression Trees (GBRT) is a generalization of boosting to arbitrary differentiable loss functions. GBRT is an accurate and effective off-the-shelf procedure that can be used for both regression and classification problems. Gradient Tree Boosting models are used in a variety of areas including Web search ranking and ecology.

In [19]:
gbc = GradientBoostingClassifier()
cv_scores = cross_val_score(gbc, x_train, y_train, cv=5)
np.mean(cv_scores)

0.97413367964459829

In [20]:
gbc2 = GradientBoostingClassifier(n_estimators=100, learning_rate=0.5,
    max_depth=10, random_state=4)
cv_scores = cross_val_score(gbc2, x_train, y_train, cv=5)
np.mean(cv_scores)

0.981199815348066

#### 4.2.2.5 Model Selection

In [21]:
from sklearn import model_selection

In [22]:
kfold = model_selection.KFold(n_splits=40, random_state=42)
estimators = []
estimators.append(('clf', clf))
estimators.append(('rfc3', rfc3))
estimators.append(('gbc2', gbc))
estimators.append(('abc', abc))
ensemble = VotingClassifier(estimators)
cv_scores = cross_val_score(ensemble, x_train, y_train, cv=5)
np.mean(cv_scores)

0.98519990607403218

In [23]:
ensemble.fit(x_train, y_train)

VotingClassifier(estimators=[('clf', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_lea...0,
            warm_start=False),
          learning_rate=1.0, n_estimators=50, random_state=None))],
         flatten_transform=None, n_jobs=1, voting='hard', weights=None)

In [24]:
ensemble.score(x_test, y_test)

0.98680000000000001

In [25]:
x_complete_train, x_complete_test, y_complete_train, y_complete_test = train_test_split(x, y, test_size=0, random_state=42)
ensemble.fit(x_complete_train, y_complete_train)

VotingClassifier(estimators=[('clf', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_lea...0,
            warm_start=False),
          learning_rate=1.0, n_estimators=50, random_state=None))],
         flatten_transform=None, n_jobs=1, voting='hard', weights=None)

#### Import real test data

In [23]:
input = pd.read_csv('data/hr_test.csv')

In [24]:
input.rename(columns={'average_montly_hours':'average_monthly_hours','Work_accident':'work_accident'},inplace=True)

In [25]:
#ID abspalten
input_ID = input['id']
input_ID_list = input_ID.tolist()
input = input.drop(['id'], axis=1)

In [26]:
# für Lösungsansatzüberflüssige Spalten entfernen
input = input.drop(['department'], axis=1)
input = input.drop(['work_accident'], axis=1)
input = input.drop(['promotion_last_5years'], axis=1)
input.head()

,satisfaction_level,last_evaluation,number_project,average_monthly_hours,time_spend_company,salary
0,0.81,0.96,4,219,2,low
1,0.86,0.84,4,246,6,low
2,0.90,0.66,4,242,3,high
3,0.37,0.54,2,131,3,medium
4,0.52,0.96,3,271,3,medium


In [27]:
input['salary_int'] = input.apply(lambda row: varianten_salary(row['salary']), axis=1)

In [28]:
input = input.drop(['salary'], axis=1)
input.head()

,satisfaction_level,last_evaluation,number_project,average_monthly_hours,time_spend_company,salary_int
0,0.81,0.96,4,219,2,1
1,0.86,0.84,4,246,6,1
2,0.90,0.66,4,242,3,3
3,0.37,0.54,2,131,3,2
4,0.52,0.96,3,271,3,2


In [38]:
prediction = abc.predict(input)

In [29]:
result = df = pd.DataFrame({'id':input_ID, 'left':prediction})
df.head()

In [ ]:
result.to_csv('data/result_v08.csv', index=False)